# Some analyses of the ID1000 dataset

In [ ]:
import imageio
import numpy as np
import pandas as pd
import joblib as jl
import nibabel as nib
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm_notebook
from nilearn import plotting, image, signal, masking
from nistats.first_level_model import FirstLevelModel
from nistats.second_level_model import SecondLevelModel
from joblib import Parallel, delayed

from scipy.interpolate import interp1d
from nistats.hemodynamic_models import glover_hrf
from scipy.io import loadmat
%matplotlib inline

Define "parameters" of fMRI data and movie stimulus

In [ ]:
t_fmri = 290 * 2.2
fps = 25
n_frames = 16500
timepoints = 220  # fmri
tr = 2.2

Let's compute luminance as a regressor for our first-level models.

In [ ]:
filename = '../ID1000_XVIDAVI_169.avi'
vid = imageio.get_reader(filename, 'ffmpeg')

lums = np.zeros((n_frames, 3))
for i, img in tqdm_notebook(enumerate(vid.iter_data())):
    # Compute luminance for fMRI design matrix
    lums[i, 0] = np.sum(img * [0.3, 0.58, 0.11])
    lums[i, 1] = np.sum(img * [0.25, 0.25, -.5])
    lums[i, 2] = np.sum(img[:, :, [0, 1]] * [0.5, -.5])

And plot it.

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=3, figsize=(15, 9))
t = np.arange(n_frames) / fps
hrf = glover_hrf(tr=1, oversampling=fps)
hrf /= hrf.max()

lum_df = pd.DataFrame()
for i in range(3):

    axes[i, 0].plot(t, lums[:, i])
    axes[i, 0].set_title("Raw luminance")

    # Convolve and downsample
    lum_conv = np.convolve(lums[:, i], hrf)[:n_frames]
    resampler = interp1d(t, lum_conv)

    # Set sample time at midpoint in TR
    t_fmri = np.linspace(tr / 2, timepoints * tr + tr / 2, 290, endpoint=False)
    lum_conv_resamp = resampler(t_fmri)

    # Standardize
    lum_conv_resamp = (lum_conv_resamp - lum_conv_resamp.mean()) / lum_conv_resamp.std()
    axes[i, 1].plot(t_fmri, lum_conv_resamp)
    axes[i, 1].set_title("HRF convolved lum")

    lum_conv_resamp_diff = np.r_[0, np.diff(lum_conv_resamp)]
    lum_conv_resamp_diff = (lum_conv_resamp_diff - lum_conv_resamp_diff.mean()) / lum_conv_resamp_diff.std()
    axes[i, 2].plot(t_fmri, lum_conv_resamp_diff)
    axes[i, 2].set_title("Diff HRF convolved lum")
    lum_df[f'lum{i}'] = lum_conv_resamp

fig.show()

Load complexity parameters and create regressors.

In [ ]:
compl = loadmat('ID1000_complexity.mat')
compl = np.c_[compl['SC'], compl['CE'], compl['BETA'], compl['GAMMA']]
compl[np.isnan(compl)] = 0
    
cols = ['SC_' + c for c in '012'] + \
       ['CE_' + c for c in '012'] + \
       ['beta_' + c for c in '012'] + \
       ['gamma_' + c for c in '012']

compl = pd.DataFrame(compl, columns=cols)

fig, axes = plt.subplots(ncols=3, nrows=4, sharex=True, figsize=(15, 10))
dm_compl = pd.DataFrame()
for i, (col, ax) in enumerate(zip(compl.columns, axes.flatten())):
    dat = compl[col].values
    dat_conv = np.convolve(dat, hrf)[:n_frames]
    resampler = interp1d(t, dat_conv)
    dat_conv_resamp = resampler(t_fmri)
    dat_conv_resamp = (dat_conv_resamp - dat_conv_resamp.mean()) / dat_conv_resamp.std()
    
    ax.plot(dat_conv_resamp)
    ax.set_title(col, fontsize=15)
    dm_compl.loc[:, col] = dat_conv_resamp

fig.tight_layout()

# Remove beta/gamma
dm_compl = dm_compl.drop(['beta_0', 'beta_1', 'beta_2', 'gamma_0', 'gamma_1', 'gamma_2'], axis=1)

In [ ]:
lum_df.index = t_fmri
dm_compl.index = lum_df.index
dm = pd.concat((lum_df, dm_compl), axis=1)
dm_compl.to_csv('design_matrix_complexity.tsv', sep='\t')
dm.corr()

In [ ]:
n_sub = 300
fmris = sorted(glob('../../derivatives/fmriprep/sub*/func/*space-MNI*bold.nii.gz'))[:n_sub]

def fit_parallel(fmri):
    conf = fmri.replace(
        'space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
        'desc-confounds_regressors.tsv'
    )
    conf = pd.read_csv(conf, sep='\t')
    cols = [col for col in conf.columns if 'cosine' in col or 'trans' in col or 'rot' in col]
    conf = conf.loc[:, cols]
    conf.index = dm.index
    dm_ = pd.concat((conf, dm), axis=1)
    dm_ = dm_.fillna(0)
    dm_['icept'] = 1
    
    flm = FirstLevelModel(
        t_r=2.2,
        mask_img=fmri.replace('preproc_bold', 'brain_mask'),
        smoothing_fwhm=5,
        minimize_memory=False
    )
    flm.fit(run_imgs=fmri, design_matrices=dm_)
    return flm

flms = Parallel(n_jobs=25)(delayed(fit_parallel)(f) for f in tqdm_notebook(fmris))
jl.dum(flms, 'models.jl')

In [ ]:
slm = SecondLevelModel()
slm.fit(flms)
img = slm.compute_contrast(first_level_contrast='lum0')
plotting.view_img(img, threshold=1.7)

In [ ]:
slm = SecondLevelModel()
slm.fit(flms)
img = slm.compute_contrast(first_level_contrast='lum1')
plotting.view_img(img, threshold=1.7)

In [ ]:
slm = SecondLevelModel()
slm.fit(flms)
img = slm.compute_contrast(first_level_contrast='lum2')
plotting.view_img(img, threshold=1.7)

In [ ]:
slm = SecondLevelModel()
slm.fit(flms)
img = slm.compute_contrast(first_level_contrast='SC_R')
plotting.view_img(img, threshold=1.7)

In [ ]:
slm = SecondLevelModel()
slm.fit(flms)
img = slm.compute_contrast(first_level_contrast='SC_R')
plotting.view_img(img, threshold=1.7)

Check TSNR.

In [ ]:
def fit_parallel_tsnr(fmri, smooth=None, remove_confounds=False):
    
    mask_img = fmri.replace('preproc_bold', 'brain_mask')
    img = image.load_img(fmri)
    if smooth is not None:
        img = image.smooth_img(img, fwhm=smooth)
    
    if remove_confounds:
        conf = fmri.replace(
            'space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
            'desc-confounds_regressors.tsv'
        )
        conf = pd.read_csv(conf, sep='\t')
        cols = [col for col in conf.columns if 'cosine' in col or 'trans' in col or 'rot' in col]
        conf = conf.loc[:, cols]
        conf = conf.fillna(0)
    
        mean_img = image.mean_img(fmri)
        ts = masking.apply_mask(fmri, mask_img)
        ts = signal.clean(ts, confounds=conf.values)
        ts += masking.apply_mask(mean_img, mask_img)
    else:
        ts = masking.apply_mask(img, mask_img)

    ts = ts.mean(axis=0) / ts.std(axis=0)
    tsnr = masking.unmask(ts, mask_img)    
    return tsnr

tsnrs = Parallel(n_jobs=25)(delayed(fit_parallel_tsnr)(f) for f in tqdm_notebook(fmris))

In [ ]:
tmp = image.mean_img(tsnrs).get_fdata()
tmp = tmp[tmp != 0]
plt.hist(tmp)

In [ ]:
plotting.view_img(image.mean_img(tsnrs), vmax=150)